In [1]:
%load_ext autoreload   

In [2]:
k = 1000
model = 'gpt-4o-mini'

In [96]:
%load_ext autoreload
%autoreload 2

import asyncio
import pandas as pd
from meeting_search import MeetingSearch, build_context_string
from typing import List

from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens

from vexa import VexaAPI
from core import system_msg, user_msg,generic_call_stream
from datetime import datetime

from prompts import Prompts
prompts = Prompts()

analyzer = MeetingSearch()
print(analyzer.count_documents())
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens, assistant_msg
from pydantic_models import QueryPlan, Query


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1279


In [97]:
async def generate_output(query, messages):
    query_ = ' '.join([m.content for m in messages])+' '+query
    queries = await analyzer.generate_search_queries(query_)
    print(queries)
    summaries =  analyzer.get_summaries()
    full_context, meeting_ids = await analyzer.build_context(queries, summaries,include_all_summaries=False)
    pref = f"Based on the following context, answer the question:" if len(messages) == 0 else "Follow-up request:"
    messages_context = [system_msg(prompts.perplexity), user_msg(f"Context:\n{full_context}")] + messages + [user_msg(f"{pref} {query}")]
    output = await generic_call_stream(messages_context)
    messages.append(user_msg(query))
    messages.append(assistant_msg(output))
    return output, messages,meeting_ids



In [122]:
messages=[]

In [127]:
query = 'now given all that create landing page'
output, messages,meeting_ids = await generate_output(query, messages)

[{'start': None, 'end': None, 'vector_search_query': 'Vexa user feedback on real-time transcription capabilities and user-friendly interface'}, {'start': None, 'end': None, 'vector_search_query': 'User experiences with Vexa compared to Fireflies and other competitors'}, {'start': None, 'end': None, 'vector_search_query': "User feedback on Vexa's absence of bots during meetings"}, {'start': None, 'end': None, 'vector_search_query': 'User-centric development feedback for Vexa'}, {'start': None, 'end': None, 'vector_search_query': "User satisfaction with Vexa's functionality and flexibility"}]
Here’s a suggested layout and content for a landing page for Vexa, incorporating user feedback and highlighting its competitive advantages:

---

## Vexa: Your Real-Time Meeting Assistant

### Transform Your Meetings with Vexa

Experience seamless communication and enhanced productivity with Vexa, the real-time transcription assistant designed for Google Meet. Say goodbye to cumbersome note-taking a

In [120]:

meeting_id_ref = 3
dashboard_url = f"https://dashboard.vexa.ai/#{meeting_ids[meeting_id_ref-1]}"
dashboard_url


'https://dashboard.vexa.ai/#c57bb398-711b-42be-aa4f-9c75b0b4759b'